In [2]:
import os 

paths = []
for dirname, _, filenames in os.walk('./data/BX-CSV-Dump'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        paths.append(os.path.join(dirname, filename))

./data/BX-CSV-Dump/BX-Book-Ratings.csv
./data/BX-CSV-Dump/BX-Books.csv
./data/BX-CSV-Dump/BX-Users.csv


In [3]:
import pandas as pd

In [4]:
ratings = pd.read_csv(paths[0], sep=';', header=0, encoding='ISO-8859-1', names=['user', 'isbn', 'rating'])
books = pd.read_csv(paths[1], sep=";", header=0, encoding="ISO-8859-1", error_bad_lines=False, usecols=[0,1,2], index_col=0, names=['isbn',"title","author"])
users = pd.read_csv(paths[2], sep=';', header=0, encoding='ISO-8859-1', names=['user', 'location', 'age'])

In [5]:
ratings.head()

,user,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
books.head()

,title,author
isbn,,
0195153448,Classical Mythology,Mark P. O. Morford
0002005018,Clara Callan,Richard Bruce Wright
0060973129,Decision in Normandy,Carlo D'Este
0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
0393045218,The Mummies of Urumchi,E. J. W. Barber


In [7]:
users.head()

,user,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


1. How many user ratings are present?

In [24]:
ratings.shape

(1149780, 3)

In [33]:
df = pd.DataFrame(ratings.user.value_counts().sort_index())
df.columns = ['number_of_ratings']
df.head()

,number_of_ratings
2,1
7,1
8,18
9,3
10,2


2. Data of how many books is there?

In [34]:
books.shape

(271379, 3)

In [35]:
books.isbn.nunique()

271379

3. Find the ratings of a particular user, say  276747

In [37]:
ratings_276747 = ratings[ratings.user == 276747]
ratings_276747

,user,isbn,rating
16,276747,0060517794,9
17,276747,0451192001,0
18,276747,0609801279,0
19,276747,0671537458,9
20,276747,0679776818,8
21,276747,0943066433,7
22,276747,1570231028,0
23,276747,1885408226,7


4. Find the ratings of user 276747 and sort them in descending order of number of ratings

In [45]:
df1 = ratings_276747.sort_values(by=['rating'], ascending=False)
df1

,user,isbn,rating
16,276747,0060517794,9
19,276747,0671537458,9
20,276747,0679776818,8
21,276747,0943066433,7
23,276747,1885408226,7
17,276747,0451192001,0
18,276747,0609801279,0
22,276747,1570231028,0


5. Find the highest 5 ratings of user 276747

In [47]:
df2 = df1[:5]
df2

,user,isbn,rating
16,276747,0060517794,9
19,276747,0671537458,9
20,276747,0679776818,8
21,276747,0943066433,7
23,276747,1885408226,7


6. Find the title and author of the favorite 5 books of user

In [61]:
for row in df2.isbn:
    print(str(row))

0060517794
0671537458
0679776818
0943066433
1885408226


In [60]:
books[books.isbn == '0679776818']

,isbn,title,author
6276,0679776818,Birdsong: A Novel of Love and War,Sebastian Faulks


In [64]:
books[books.isbn == '0679776818'].title.values

array(['Birdsong: A Novel of Love and War'], dtype=object)

In [66]:
str(books[books.isbn == '0679776818'].title.values)

"['Birdsong: A Novel of Love and War']"

In [69]:
for row in df2.isbn:
    row = str(row)
    print('Title:', str(books[books.isbn == row].title.values))
    print('Author:', str(books[books.isbn == row].author.values))

Title: ['Little Altars Everywhere']
Author: ['Rebecca Wells']
Title: ['Waiting to Exhale']
Author: ['Terry McMillan']
Title: ['Birdsong: A Novel of Love and War']
Author: ['Sebastian Faulks']
Title: ['How to Deal With Difficult People']
Author: ['Rick Brinkman']
Title: ['The Golden Rule of Schmoozing']
Author: ['Aye Jaye']


7. Find the number of books (isbns) rated per user

In [70]:
ratings.head()

,user,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [71]:
ratings.user.value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
158698        1
17920         1
277135        1
275086        1
187812        1
Name: user, Length: 105283, dtype: int64

8. Find the number of users who have rated each book(isbn)

In [8]:
ratings['isbn'].isin(books.index)

0           True
1           True
2           True
3           True
4           True
           ...  
1149775     True
1149776     True
1149777     True
1149778     True
1149779    False
Name: isbn, Length: 1149780, dtype: bool

In [11]:
cleaned_ratings = ratings[ratings['isbn'].isin(books.index)]
cleaned_ratings.head()

,user,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [13]:
cleaned_ratings.shape, ratings.shape

((1031175, 3), (1149780, 3))

In [14]:
1149780 - 1031175

118605

In [15]:
pd.pivot_table(cleaned_ratings, values='rating', index=['user'], columns=['isbn'])

ValueError: Unstacked DataFrame is too big, causing int32 overflow

In [18]:
UsersPerISBN = cleaned_ratings.isbn.value_counts()
UsersPerISBN

0971880107    2502
0316666343    1295
0385504209     883
0060928336     732
0312195516     723
              ... 
0809232103       1
3405131138       1
1931484759       1
013601013X       1
0819308072       1
Name: isbn, Length: 270170, dtype: int64

In [19]:
ISBNsPerUser = cleaned_ratings.user.value_counts()
ISBNsPerUser

11676     11144
198711     6456
153662     5814
98391      5779
35859      5646
          ...  
82220         1
94506         1
67877         1
78114         1
145222        1
Name: user, Length: 92107, dtype: int64

In [20]:
UsersPerISBN[UsersPerISBN > 10]

0971880107    2502
0316666343    1295
0385504209     883
0060928336     732
0312195516     723
              ... 
0440236908      11
0440212685      11
0373290772      11
0812533593      11
0517575019      11
Name: isbn, Length: 15452, dtype: int64

In [22]:
cleaned_ratings.isbn.isin(UsersPerISBN[UsersPerISBN > 10].index)

0           True
1          False
2           True
3          False
4          False
           ...  
1149774    False
1149775    False
1149776     True
1149777     True
1149778     True
Name: isbn, Length: 1031175, dtype: bool

In [24]:
data = cleaned_ratings[cleaned_ratings.isbn.isin(UsersPerISBN[UsersPerISBN > 10].index)]
data.head()

,user,isbn,rating
0,276725,034545104X,0
2,276727,0446520802,0
8,276744,038550120X,7
10,276746,0425115801,0
11,276746,0449006522,0


In [25]:
data.shape

(500229, 3)

In [26]:
data = cleaned_ratings[cleaned_ratings.user.isin(ISBNsPerUser[ISBNsPerUser > 10])]
data.head()

,user,isbn,rating
9586,12,1879384493,10
9587,14,0061076031,5
9588,14,0439095026,5
9589,14,0689821166,6
9590,14,0971880107,0


In [27]:
data.shape

(1072, 3)

In [29]:
pivot_table = pd.pivot_table(data, values='rating', index=['user'], columns=['isbn'])
pivot_table.head()

isbn,0002740230,0006276199,000649840X,0007100221,0020437501,006000438X,0060013117,0060168013,0060173289,0060177586,...,8483002523,848300402X,8483007010,8486433193,8486433770,8807120194,8845229041,887641486X,9681500555,9724119378
user,,,,,,,,,,,,,,,,,,,,,
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
pivot_table.dropna()

isbn,0002740230,0006276199,000649840X,0007100221,0020437501,006000438X,0060013117,0060168013,0060173289,0060177586,...,8483002523,848300402X,8483007010,8486433193,8486433770,8807120194,8845229041,887641486X,9681500555,9724119378
user,,,,,,,,,,,,,,,,,,,,,
